<a href="https://colab.research.google.com/github/miloszpiechota/AI-Systems/blob/main/PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('data.csv')
df.head()
df.drop('Tm',axis=1,inplace=True)
X=df.iloc[:,0:27]
y = df.iloc[:,27]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=21)
scaleStandard = StandardScaler()
X_train = scaleStandard.fit_transform(X_train)
X_test = scaleStandard.transform(X_test)
df.columns

X_train=pd.DataFrame(X_train,columns=['A','B','C'])

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
pca1=PCA()
X_pca1 = pca1.fit_transform(X_train)
pca1.explained_variance_ratio_
plt.plot(pca1.explained_variance_ratio_)
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')
plt.show()

pca2=PCA(0.95)
X_pca2 = pca2.fit_transform(X_train)
X_pca2.shape
pca2.explained_variance_ratio_

pca2c = PCA(n_components=2)
X_pca2c = pca2c.fit_transform(X_train)

colormap=plt.cm.get_cmap('coolwarm')
plt.figure()
scatter = plt.scatter(X_pca2c[:,0],X_pca2c[:,1],c=y_train,cmap=colormap)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.colorbar(scatter)
plt.show()

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold
from scipy import stats
import os

# Define file paths
feature_names_file = 'features.txt'
activity_labels_file = 'activity_labels.txt'
train_data_file = 'train/X_train.txt'
train_labels_file = 'train/y_train.txt'
train_subjects_file = 'train/subject_train.txt'
test_data_file = 'test/X_test.txt'
test_labels_file = 'test/y_test.txt'
test_subjects_file = 'test/subject_test.txt'

# Load feature names
feature_names = pd.read_csv(feature_names_file, delim_whitespace=True, header=None, usecols=[1])
feature_names = feature_names[1].tolist()

# Load activity labels
activity_labels = pd.read_csv(activity_labels_file, delim_whitespace=True, header=None, index_col=0)
activity_labels_dict = activity_labels[1].to_dict()

# Load training data
X_train = pd.read_csv(train_data_file, delim_whitespace=True, header=None)
y_train = pd.read_csv(train_labels_file, delim_whitespace=True, header=None)
subject_train = pd.read_csv(train_subjects_file, delim_whitespace=True, header=None)

# Load test data
X_test = pd.read_csv(test_data_file, delim_whitespace=True, header=None)
y_test = pd.read_csv(test_labels_file, delim_whitespace=True, header=None)
subject_test = pd.read_csv(test_subjects_file, delim_whitespace=True, header=None)

# Combine training and test data
X = pd.concat([X_train, X_test], ignore_index=True)
y = pd.concat([y_train, y_test], ignore_index=True)
subjects = pd.concat([subject_train, subject_test], ignore_index=True)

# Assign column names
X.columns = feature_names
y.columns = ['Activity']
subjects.columns = ['Subject']

# Map activity labels
y['Activity'] = y['Activity'].map(activity_labels_dict)

# Combine all data into a single DataFrame
data = pd.concat([subjects, y, X], axis=1)

# Handle missing values (if any)
data.dropna(inplace=True)

# Remove outliers using Z-score
z_scores = np.abs(stats.zscore(data.iloc[:, 2:]))
data = data[(z_scores < 3).all(axis=1)]

# Remove low-variance features
selector = VarianceThreshold(threshold=0.01)
X_reduced = selector.fit_transform(data.iloc[:, 2:])
selected_features = data.iloc[:, 2:].columns[selector.get_support(indices=True)]
data_reduced = pd.DataFrame(X_reduced, columns=selected_features)
data = pd.concat([data.iloc[:, :2].reset_index(drop=True), data_reduced], axis=1)

# Remove highly correlated features
corr_matrix = data.iloc[:, 2:].corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]
data.drop(columns=to_drop, inplace=True)

# Standardize the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(data.iloc[:, 2:])
data_scaled = pd.DataFrame(features_scaled, columns=data.columns[2:])
data_scaled = pd.concat([data.iloc[:, :2].reset_index(drop=True), data_scaled], axis=1)

# Save the cleaned and standardized data
data_scaled.to_csv('cleaned_data.csv', index=False)

# Apply PCA to retain 95% of variance
pca = PCA(n_components=0.95, random_state=42)
principal_components = pca.fit_transform(data_scaled.iloc[:, 2:])
pca_columns = [f'PC{i+1}' for i in range(principal_components.shape[1])]
pca_data = pd.DataFrame(principal_components, columns=pca_columns)
pca_data = pd.concat([data_scaled.iloc[:, :2].reset_index(drop=True), pca_data], axis=1)

# Save the PCA-transformed data
pca_data.to_csv('pca_data.csv', index=False)

print("Data preprocessing complete. 'cleaned_data.csv' and 'pca_data.csv' have been saved.")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')